# Implementation verification: Toxicokinetics

In [18]:
using Revise
@time using DEBBase
using Plots, Plots.Measures, StatsPlots
using DataFrames
using BenchmarkTools

ERROR: LoadError: 

UndefVarError: `S` not defined


Stacktrace:
  [1] DEB!(du::Vector{Float64}, u::Vector{Float64}, p::NamedTuple{(:glb, :deb), Tuple{DEBBase.GlobalBaseParams, DEBBase.DEBBaseParams}}, t::Float64)
    @ DEBBase c:\Users\simon\Documents\Julia\DEBBase.jl\src\ModelFunctions.jl:209
  [2] (::SciMLBase.Void{typeof(DEBBase.DEB!)})(::Vector{Float64}, ::

Vararg{Any})
    @ SciMLBase C:\Users\simon\.julia\packages\SciMLBase\8XHkk\src\utils.jl:481
  [3] (::FunctionWrappers.CallWrapper{Nothing})(f::SciMLBase.Void{typeof(DEBBase.DEB!)}, arg1::Vector{Float64}, arg2::Vector{Float64}, arg3::NamedTuple{(:glb, :deb), Tuple{DEBBase.GlobalBaseParams, DEBBase.DEBBaseParams}}, arg4::Float64)
    @ FunctionWrappers C:\Users\simon\.julia\packages\FunctionWrappers\Q5cBx\src\FunctionWrappers.jl:65
  [4] macro expansion
    @ C:\Users\simon\.julia\packages\FunctionWrappers\Q5cBx\src\FunctionWrappers.jl:137 [inlined]
  [5] do_ccall
    @ C:\Users\simon\.julia\packages\FunctionWrappers\Q5cBx\src\FunctionWrappers.jl:125 [inlined]
  [6] FunctionWrapper
    @ C:\Users\simon\.julia\packages\FunctionWrappers\Q5cBx\src\FunctionWrappers.jl:144 [inlined]
  [7] _call
    @ C:\Users\simon\.julia\packages\FunctionWrappersWrappers\9XR0m\src\FunctionWrappersWrappers.jl:12 [inlined]
  [8] FunctionWrappersWrapper
    @ C:\Users\simon\.julia\packages\FunctionWrappersWrap

OrdinaryDiffEq.ODEIntegrator{OrdinaryDiffEq.CompositeAlgorithm{Tuple{OrdinaryDiffEq.Vern7{typeof(OrdinaryDiffEq.trivial_limiter!), typeof(OrdinaryDiffEq.trivial_limiter!), Static.False}, OrdinaryDiffEq.Rodas5P{1, false, Nothing, typeof(OrdinaryDiffEq.DEFAULT_PRECS), Val{:forward}, true, nothing}}, OrdinaryDiffEq.AutoSwitchCache{OrdinaryDiffEq.Vern7{typeof(OrdinaryDiffEq.trivial_limiter!), typeof(OrdinaryDiffEq.trivial_limiter!), Static.False}, OrdinaryDiffEq.Rodas5P{0, false, Nothing, typeof(OrdinaryDiffEq.DEFAULT_PRECS), Val{:forward}, true, nothing}, Rational{Int64}, Int64}}, true, Vector{Float64}, Nothing, Float64, NamedTuple{(:glb, :deb), Tuple{DEBBase.GlobalBaseParams, DEBBase.DEBBaseParams}}, Float64, Float64, Float64, Float64, Vector{Vector{Float64}}, SciMLBase.ODESolution{Float64, 2, Vector{Vector{Float64}}, Nothing, Nothing, Vector{Float64}, Vector{Vector{Vector{Float64}}}, SciMLBase.ODEProblem{Vector{Float64}, Tuple{Float64, Float64}, true, NamedTuple{(:glb, :deb), Tuple{DEBB

...

OrdinaryDiffEq.ODEIntegrator{OrdinaryDiffEq.CompositeAlgorithm{Tuple{OrdinaryDiffEq.Vern7{typeof(OrdinaryDiffEq.trivial_limiter!), typeof(OrdinaryDiffEq.trivial_limiter!), Static.False}, OrdinaryDiffEq.Rodas5P{1, false, Nothing, typeof(OrdinaryDiffEq.DEFAULT_PRECS), Val{:forward}, true, nothing}}, OrdinaryDiffEq.AutoSwitchCache{OrdinaryDiffEq.Vern7{typeof(OrdinaryDiffEq.trivial_limiter!), typeof(OrdinaryDiffEq.trivial_limiter!), Static.False}, OrdinaryDiffEq.Rodas5P{0, false, Nothing, typeof(OrdinaryDiffEq.DEFAULT_PRECS), Val{:forward}, true, nothing}, Rational{Int64}, Int64}}, true, Vector{Float64}, Nothing, Float64, NamedTuple{(:glb, :deb), Tuple{DEBBase.GlobalBaseParams, DEBBase.DEBBaseParams}}, Float64, Float64, Float64, Float64, Vector{Vector{Float64}}, SciMLBase.ODESolution{Float64, 2, Vector{Vector{Float64}}, Nothing, Nothing, Vector{Float64}, Vector{Vector{Vector{Float64}}}, SciMLBase.ODEProblem{Vector{Float64}, Tuple{Float64, Float64}, true, NamedTuple{(:glb, :deb), Tuple{DEBB


    @ OrdinaryDiffEq C:\Users\simon\.julia\packages\OrdinaryDiffEq\Y6bIX\src\solve.jl:565
 [20] __init(prob::SciMLBase.ODEProblem{Vector{Float64}, Tuple{Float64, Float64}, true, NamedTuple{(:glb, :deb), Tuple{DEBBase.GlobalBaseParams, DEBBase.DEBBaseParams}}, SciMLBase.ODEFunction{true, SciMLBase.AutoSpecialize, FunctionWrappersWrappers.FunctionWrappersWrapper{Tuple{FunctionWrappers.FunctionWrapper{Nothing, Tuple{Vector{Float64}, Vector{Float64}, NamedTuple{(:glb, :deb), Tuple{DEBBase.GlobalBaseParams, DEBBase.DEBBaseParams}}, Float64}}, FunctionWrappers.FunctionWrapper{Nothing, Tuple{Vector{ForwardDiff.Dual{ForwardDiff.Tag{DiffEqBase.OrdinaryDiffEqTag, Float64}, Float64, 1}}, Vector{ForwardDiff.Dual{ForwardDiff.Tag{DiffEqBase.OrdinaryDiffEqTag, Float64}, Float64, 1}}, NamedTuple{(:glb, :deb), Tuple{DEBBase.GlobalBaseParams, DEBBase.DEBBaseParams}}, Float64}}, FunctionWrappers.FunctionWrapper{Nothing, Tuple{Vector{ForwardDiff.Dual{ForwardDiff.Tag{DiffEqBase.OrdinaryDiffEqTag, Float64}

; merge_callbacks::Bool, kwargshandle::Nothing, kwargs::Base.Pairs{Symbol, Real, NTuple{4, Symbol}, NamedTuple{(:default_set, :second_time, :reltol, :abstol), Tuple{Bool, Bool, Float64, Float64}}})
    @ DiffEqBase C:\Users\simon\.julia\packages\DiffEqBase\nKsvb\src\solve.jl:605
 [25] solve_up(prob::SciMLBase.ODEProblem{Vector{Float64}, Tuple{Float64, Float64}, true, NamedTuple{(:glb, :deb), Tuple{DEBBase.GlobalBaseParams, DEBBase.DEBBaseParams}}, SciMLBase.ODEFunction{true, SciMLBase.AutoSpecialize, FunctionWrappersWrappers.FunctionWrappersWrapper{Tuple{FunctionWrappers.FunctionWrapper{Nothing, Tuple{Vector{Float64}, Vector{Float64}, NamedTuple{(:glb, :deb), Tuple{DEBBase.GlobalBaseParams, DEBBase.DEBBaseParams}}, Float64}}, FunctionWrappers.FunctionWrapper{Nothing, Tuple{Vector{ForwardDiff.Dual{ForwardDiff.Tag{DiffEqBase.OrdinaryDiffEqTag, Float64}, Float64, 1}}, Vector{ForwardDiff.Dual{ForwardDiff.Tag{DiffEqBase.OrdinaryDiffEqTag, Float64}, Float64, 1}}, NamedTuple{(:glb, :deb), Tup

)
    @ DiffEqBase C:\Users\simon\.julia\packages\DiffEqBase\nKsvb\src\solve.jl:1046
 [34] solve_up
    @ C:\Users\simon\.julia\packages\DiffEqBase\nKsvb\src\solve.jl:1040 [inlined]
 [35] solve(::SciMLBase.ODEProblem{Vector{Float64}, Tuple{Float64, Float64}, true, NamedTuple{(:glb, :deb), Tuple{DEBBase.GlobalBaseParams, DEBBase.DEBBaseParams}}, SciMLBase.ODEFunction{true, SciMLBase.AutoSpecialize, typeof(DEBBase.DEB!), LinearAlgebra.UniformScaling{Bool}, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, typeof(SciMLBase.DEFAULT_OBSERVED), Nothing, Nothing}, Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}, SciMLBase.StandardODEProblem}; sensealg::Nothing, u0::Nothing, p::Nothing, wrap::Val{true}, kwargs::Base.Pairs{Symbol, Float64, Tuple{Symbol, Symbol}, NamedTuple{(:reltol, :abstol), Tuple{Float64, Float64}}})
    @ DiffEqBase C:\Users\simon\.julia\packages\DiffEqBase\nKsvb\src\solve.jl:977
 [36] 

ErrorException: Failed to precompile DEBBase [0e91d03b-6562-4ef9-a38e-775ced00eaf8] to "C:\\Users\\simon\\.julia\\compiled\\v1.9\\DEBBase\\jl_BC50.tmp".

In [7]:
glb = GlobalBaseParams()

UndefVarError: UndefVarError: `GlobalBaseParams` not defined

In [15]:
u = [1., [2., 3.]]
statevars = [:X, :Y]
sttypes = [Float64, Vector{Float64}]

for (i, u_i) in enumerate(u)
    eval(:($(statevars[i]) = $(sttypes[i])($(u_i))))
end

In [16]:
Y

2-element Vector{Float64}:
 2.0
 3.0

## Simulating TK for a single stressor

## Varying the number of stressors

## Varying coupled TK parameter

## Uncoupling the TK parameter

## Modifying the model: Simplified TK

We start by having a look at the predefined TK function:

In [39]:
less(DEBBase.Ddot!)

function Ddot!(
    deb::AbstractParams,
    z::Int64; 
    C_W::Float64, 
    D::Float64, 
    S::Float64, 
    Sdot::Float64, 
    S_max::Float64
    )
    # surface area / volume ratio
    let AV_ratio = S^(2/3) / S, AV_max_ratio = S_max^(2/3) / S_max
        # TK with account for body size: smaller A/V ration = higher elimination rate, diluation by growth
        return deb.k_D[z] * (AV_max_ratio / AV_ratio) * (C_W[z] - D[z]) - D[z] * (Sdot/S)
    end
end

"""
Definition of reserveless DEB derivatives. 
$(TYPEDSIGNATURES)
"""
function DEB!(du, u, p, t)
    glb, deb = p
    # unpack scalar state variables
    X_p, X_emb, S, H, R = u

    C_W = u[glb.num_scalar_statevars+1:Int(glb.num_scalar_statevars+(length(u)-glb.num_scalar_statevars)/glb.num_vector_statevars)]
    D = u[Int(glb:num_scalar_statevars+1+length(C_W)):end]

    S = max(0, S) # control for negative values

    life_stage = determine_life_stage(deb; H = H, X_emb = X_emb)
    
    ddot 
    for (z,C_Wz) in enumerate(C_W)

In [30]:
function DEBBase.Ddot!()

end

In [45]:
less(DEBBaseParams())

MethodError: MethodError: no method matching DEBBaseParams()

In [47]:
[Float64, Int64]

2-element Vector{DataType}:
 Float64
 Int64

In [75]:
statevars = [:X_p, :X_emb, :S, :C_W]
sttypes = [Float64, Float64, Float64, Vector{Float64}]
u = [0.33, 2., 3., [4., 5., 314.]]

4-element Vector{Any}:
 0.33
 2.0
 3.0
  [4.0, 5.0, 314.0]

In [79]:
using DEBBase

In [81]:
DEBBase.@unpack u 

LoadError: LoadError: MethodError: no method matching var"@unpack"(::LineNumberNode, ::Module)

Closest candidates are:
  var"@unpack"(::LineNumberNode, ::Module, !Matched::Any)
   @ UnPack C:\Users\simon\.julia\packages\UnPack\EkESO\src\UnPack.jl:92

in expression starting at c:\Users\simon\Documents\Julia\DEBBase.jl\notebooks\ImplementationVerification\ImplemenationVerification_20231222_TK.ipynb:1

In [77]:
@unpack u

In [78]:
C_W

3-element Vector{Float64}:
   4.0
   5.0
 314.0

In [41]:
less(DEBBaseParams())

MethodError: MethodError: no method matching DEBBaseParams()